In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
df = pd.read_csv("../input/train.csv")
df.sample(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
28612,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,234,253,208,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,117,253,130,0,0,0,0,0,0,0
10382,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,191,41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
y = df.iloc[:,:1]
x = df.iloc[:,1:]

# Input data

In [4]:
from math import ceil

n = x.shape[0]
n_train = ceil(0.8 * n)
n_test = ceil(0.2 * n)

x_train = x[:n_train]
y_train = y[:n_train]

x_test = x[n_train:]
y_test = y[n_train:]

y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

# Building the model and evaluating

In [5]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(solver='adam', 
                          alpha=1e-5, 
                          hidden_layer_sizes=(1024, 128), 
                          random_state=1, 
                          max_iter=150)

print(mlp_model)

mlp_model.fit(x_train, y_train)                         
pred_mlp =  mlp_model.predict(x_test)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1024, 128), learning_rate='constant',
       learning_rate_init=0.001, max_iter=150, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)


In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics

print("\n\nTesting using MLP")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, pred_mlp))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, pred_mlp))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred_mlp))) 



Testing using MLP
Mean Absolute Error: 0.09892857142857144
Mean Squared Error: 0.5091666666666667
Root Mean Squared Error: 0.7135591542869215


In [7]:
print("\n\nMLP")
print("--- confusion_matrix ---")
print(confusion_matrix(y_test, pred_mlp))  
print("\n--- classification report ---")
print(classification_report(y_test, pred_mlp))  
print("\nmodel accuracy: ", accuracy_score(y_test, pred_mlp))



MLP
--- confusion_matrix ---
[[848   0   1   0   0   0   2   0   1   0]
 [  1 933   5   1   1   1   2   3   3   0]
 [  3   1 784   3   1   1   1   2   3   2]
 [  1   0   7 859   0   6   0   3   3   3]
 [  0   5   1   0 788   0   4   2   2   9]
 [  2   0   1  11   0 715   8   1   3   6]
 [  3   1   3   0   3   0 821   0   0   0]
 [  1   1   3   1   1   1   0 871   0   5]
 [  6   7   7   7   3   4   4   1 761   2]
 [  4   3   2   3   9   0   0  10   1 808]]

--- classification report ---
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       852
           1       0.98      0.98      0.98       950
           2       0.96      0.98      0.97       801
           3       0.97      0.97      0.97       882
           4       0.98      0.97      0.97       811
           5       0.98      0.96      0.97       747
           6       0.98      0.99      0.98       831
           7       0.98      0.99      0.98       884
           8       0

# Recognizing numbers (or not)

In [8]:
df = pd.read_csv("../input/test.csv")
df.sample(2)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1606,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13707,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
test = df.values
test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
preds =  mlp_model.predict(test)

In [11]:
index = df.index + 1
d = {"ImageId" : index, "Label" : preds}
df_out = pd.DataFrame(data=d)
df_out.sample(1)

,ImageId,Label
19870,19871,2


In [12]:
df_out.to_csv("rec_numbers.csv", index=False)